In [1]:
import os
import gradio as gr
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import geopandas as gp
import utm
import json
import shapely
from urllib.request import urlopen
from PIL import Image

In [2]:
df = gp.read_file(r'C:\Users\jairl\Documents\GitHub\CORE\website\dataframe.geojson')
def latlon_from_utm(x):
    _coords = utm.to_latlon(x['geometry'].centroid.x,x['geometry'].centroid.y,zone_number=37,northern=True)
    return(shapely.Point(_coords))

df['latlon']=df.apply(latlon_from_utm,1)
df['id'] = np.arange(len(df))

In [3]:
with open('turkey_704.geojson') as file:
    districts_json = json.load(file)
districts = gp.read_file('turkey_704.geojson',encoding='')

In [4]:
districts['num_buildings'] = 0 
_coords = df.latlon.apply(lambda geom: shapely.ops.transform(lambda x, y: (y, x), geom))
for n,district in enumerate(districts.geometry):
    
    n_buildings = sum(district.contains(_coords))
    
    if n_buildings != 0:
        districts.loc[n,'num_buildings'] = n_buildings
    else: 
        districts.loc[n,'num_buildings'] = np.random.randint(0,1000)

In [5]:
df['material'] = 'wood'
df['occupancy'] = np.random.randint(1,7,len(df))
df['area'] = round(df['geometry'].area)

In [6]:
destroyed = df[df['dmg']==4]
not_destroyed = df[df['dmg']!=4]

In [7]:
layout = go.Layout(autosize=False,width=1000,height=600)
fig = go.Figure(layout=layout)

fig.add_trace(go.Scattermapbox(
        lat=not_destroyed['latlon'].x,
        lon=not_destroyed['latlon'].y,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=2,
            sizemin=15,
            color='white',
            opacity=1
        ),
        hoverinfo='none'
    ))


fig.add_trace(go.Scattermapbox(
            customdata=destroyed[['occupancy','area','material','id']].values.tolist(),
            lat=destroyed['latlon'].x,
            lon=destroyed['latlon'].y,
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=10,
                opacity = 1,
                color = 'red',
            ),
            hovertemplate='<b>ID</b>: %{customdata[3]}<br><b>Expected occupancy</b>: %{customdata[0]}<br><b>Area</b>: %{customdata[1]}m<sup>2</sup><br><b>Contruction material</b>: %{customdata[2]}',
            ),
            )

fig.update_layout(
    mapbox_style="open-street-map",
    hovermode='closest',
    hoverdistance =1000,
    mapbox=dict(
        bearing=0,
        center=go.layout.mapbox.Center(
            lat= 37,
            lon=36
        ),
        pitch=5,
        zoom=7
    
    )
)
with gr.Blocks() as regionmap:
    map = gr.Plot(fig,scale=2,container=False)
    regionmap.launch(server_port=7880) 

Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


In [8]:
images = []
for image_path in os.listdir('./folder'):
    images.append(Image.open(f'./folder/{image_path}'))
destroyed[['image']] = images[:len(destroyed)]

c:\Users\jairl\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
def get_image(id):
    return(destroyed.loc[int(id)].image.resize((500,500)))
def get_data(id):
    _data = destroyed.loc[id][['id','occupancy','area','material']]
    html = '<b>Occupancy</b>: %i <b>Area</b>: %im<sup>2</sup> <b>Primary construction material</b>: %s' % (_data[1],_data[2],_data[3])
    return(html)

data = destroyed.iloc[0][['latlon','occupancy','area','material']]
with gr.Blocks() as building_information:
    with gr.Column():
        id = gr.Dropdown(label='Building ID',choices=destroyed.id.tolist(),value=destroyed.id.iloc[0])
        info = gr.HTML(value=('<b>Occupancy</b>: %i <b>Area</b>: %im<sup>2</sup> <b>Primary construction material</b>: %s' % (data[1],data[2],data[3])))
        image = gr.Image(value=destroyed.image.iloc[0].resize((500,500)),scale=1)
        id.select(get_image,inputs=[id], outputs=[image])
        id.select(get_data,inputs=[id], outputs=[info])
building_information.launch(server_port=7881)

Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


In [10]:
worldmap_fig = go.Figure(layout=layout)

worldmap_fig.add_trace(go.Choroplethmapbox(geojson=districts_json, 
                                    locations=np.arange(1,len(districts)+1).tolist(), 
                                    z=districts.num_buildings,
                                    customdata=districts[['name','num_buildings']].values.tolist(),
                                    colorscale=[[0, 'rgb(255,255,255)'],[1, 'rgb(255,0,0)']], zauto=True,
                                    marker_opacity=0.5, 
                                    marker_line_width=1, 
                                    hovertemplate='Province: <a href="https://drive.google.com/file/d/1A7bXvvn6Np7uQhXjGzSDeH-bGsdZmtsY/view?usp=drive_link" target="_top"> %{customdata[0]}</a><br>Num damaged: %{customdata[1]}'))

worldmap_fig.update_layout(
    mapbox_style="open-street-map",
    hovermode='closest',
    mapbox=dict(
        bearing=0,
        center=go.layout.mapbox.Center(
            lat= 37,
            lon=36
        ),
        pitch=5,
        zoom=3
    )
)


with gr.Blocks() as worldmap:
    map = gr.Plot(worldmap_fig)

    
worldmap.launch(server_port=7882)   

Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.
